In [ ]:
video_ID = "Y7t5B69G0Dw"
API_key = "API_KEY"

In [ ]:
import requests
from datetime import datetime, timezone

def get_channel_subscriber_count(channel_id, api_key=API_key):
    url = "https://www.googleapis.com/youtube/v3/channels"
    params = {
        "part": "statistics",
        "id": channel_id,
        "key": api_key
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()

    if "items" in data and len(data["items"]) > 0:
        return int(data["items"][0]["statistics"].get("subscriberCount"))
    return None


def get_video_upload_time(video_id, api_key):
    """Returns the video's upload datetime object (UTC)."""
    url = "https://www.googleapis.com/youtube/v3/videos"
    params = {
        "part": "snippet",
        "id": video_id,
        "key": api_key
    }
    resp = requests.get(url, params=params).json()
    upload_time_str = resp['items'][0]['snippet']['publishedAt']
    return datetime.strptime(upload_time_str, "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc)


def get_all_youtube_comments(video_id=video_ID, api_key=API_key):
    video_upload_time = get_video_upload_time(video_id, api_key)
    comments = []
    url = "https://www.googleapis.com/youtube/v3/commentThreads"
    params = {
        "part": "snippet",
        "videoId": video_id,
        "key": api_key,
        "maxResults": 100,  # max per page
        "textFormat": "plainText"
    }

    next_page_token = None
    while True:
        if next_page_token:
            params["pageToken"] = next_page_token

        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        for item in data.get("items", []):
            snippet = item["snippet"]["topLevelComment"]["snippet"]

            comment_time = datetime.strptime(snippet["publishedAt"], "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc)
            updated_time = datetime.strptime(snippet["updatedAt"], "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc)

            seconds_since_upload = int((comment_time - video_upload_time).total_seconds())
            seconds_since_update = int((updated_time - video_upload_time).total_seconds())

            likes = snippet.get("likeCount", 0)
            if likes > 100:
              comments.append({
                    "author": snippet.get("authorDisplayName"),
                    "comment": snippet.get("textDisplay"),
                    "likes": likes,
                    "replies": item["snippet"].get("totalReplyCount", 0),
                    "timestamp": seconds_since_upload,
                    "updatedTimestamp": seconds_since_update,
                    "subscriberCount": get_channel_subscriber_count(snippet.get("authorChannelId")['value']),
              })
            else:
              comments.append({
                    "author": snippet.get("authorDisplayName"),
                    "comment": snippet.get("textDisplay"),
                    "likes": likes,
                    "replies": item["snippet"].get("totalReplyCount", 0),
                    "timestamp": seconds_since_upload,
                    "updatedTimestamp": seconds_since_update,
                    "subscriberCount": -1,
              })

        next_page_token = data.get("nextPageToken")
        if not next_page_token:
            break

    # Optional: sort by likes descending
    comments.sort(key=lambda x: x["likes"], reverse=True)

    return comments

comments = get_all_youtube_comments()


In [ ]:
print(len(comments))

126043


In [ ]:
print(comments[0])

{'author': '@Joe-sw3lp', 'comment': 'dream should do a manhunt with 5 hunters but one of them is actually a traitor and is on dreams side trying to sabotage the hunters.', 'likes': 64247, 'replies': 432, 'timestamp': 107629, 'updatedTimestamp': 107629, 'subscriberCount': 19}


In [ ]:
import csv

video_id = video_ID

# Get CSV headers from the first comment
headers = comments[0].keys() if comments else []

with open(video_id + ".csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=headers)
    writer.writeheader()
    writer.writerows(comments)

print("CSV saved as youtube_comments.csv")

CSV saved as youtube_comments.csv
